
# Chapter 14 最小二乗識別

## 14.1 識別

### 真偽値

Juliaの真偽値は`true`と`false`である．これらは数値を含む式では自動的に1と0に変換される．しかし本書では，2クラス分類のために真を$+1$に，偽を$-1$に置き換えている．これをJuliaの真偽値`b`で行うには`2*b-1`とするか，もしくは三項演算子を使って`b ? 1 : -1`とすればよい．

In [1]:
tf2pm1(b) = 2 * b - 1;

In [2]:
b = true

true

In [3]:
tf2pm1(b)

1

In [4]:
b = false

false

In [5]:
tf2pm1(b)

-1

In [6]:
b = [ true, false, true ]

3-element Array{Bool,1}:
 1
 0
 1

In [7]:
tf2pm1.(b)

3-element Array{Int64,1}:
  1
 -1
  1

### 混同行列

データ`y`と予測`ypred`が長さ`N`の真偽値の配列（ベクトル）で与えられた場合，
予測誤り率と混同行列の計算は以下のようになる．

In [12]:
# 正しい予測と誤った予測を加須れる
Ntp(y, yhat) = sum( (y .== true) .& (yhat .== true) );
Nfn(y, yhat) = sum( (y .== true) .& (yhat .== false) );
Nfp(y, yhat) = sum( (y .== false) .& (yhat .== true) );
Ntn(y, yhat) = sum( (y .== false) .& (yhat .== false) );
error_rate(y, yhat) = (Nfn(y, yhat) + Nfp(y, yhat)) / length(y);
confusion_matrix(y,yhat) = [ Ntp(y, yhat) Nfn(y, yhat);
                             Nfp(y, yhat) Ntn(y, yhat) ];
y = rand(Bool, 100);
yhat = rand(Bool, 100);

In [13]:
confusion_matrix(y, yhat)

2×2 Array{Int64,2}:
 30  14
 28  28

In [14]:
error_rate(y, yhat)

0.42

`==`と`&`の前のドットは要素毎の演算であることを意味する．真偽値のベクトルの総和を計算しようとすると，真偽値は整数に変換される．コードの最後の方では，2つのランダムな真偽値ベクトルを生成しているので，誤り率は50\%程度のはずである．このコードでは，偽陰性と偽陽性の誤りから誤り率を求めている．もっと簡潔に誤り率を求めるなら`avg(y .!= yhat)`と書く．`VMLS`パッケージには関数`confusion_matrix(y, yhat)`が含まれている．


## 14.2 最小二乗識別器

$\hat{f}(x) = \mathbf{sign}(\tilde{f}(x))$を評価するには，以下で定義する関数を使って`ftilde(x) > 0`とする．これは真偽値を返す．

```julia
julia> ftilde(x) = x' * beta .+ v  # 回帰モデル
julia> fhat(x) = ftilde(x) > 0  # 識別器
```

### iris flower識別問題

Irisデータセットには，3種類のアヤメ150サンプルが含まれている．各クラス50サンプルである．各サンプルは4つの特徴量からなる．以下のコードは3つのクラスのサンプルを保持する$50 \times 4$行列`setosa`，`versicolor`，`virginica`を含む辞書を読み込み，Iris Virginicaとそれ以外のクラスを識別する二値識別器を学習する．

In [15]:
using VMLS

In [16]:
D = iris_data();
# 150 x 4 データ行列を作成
iris = vcat(D["setosa"], D["versicolor"], D["virginica"]);
# virginicaならy[k]はtrue (1)，そうでなければfalse (0)
y = [ zeros(Bool, 50); zeros(Bool, 50); ones(Bool, 50) ];
A = [ ones(150) iris ];

In [17]:
theta = A \ (2 * y .- 1)

5-element Array{Float64,1}:
 -2.3905637266512043  
 -0.0917521691013458  
  0.4055367711191057  
  0.007975822012793829
  1.1035586498675736  

In [18]:
yhat = A * theta .> 0;

In [19]:
C = confusion_matrix(y, yhat)

2×2 Array{Int64,2}:
 46   4
  7  93

In [17]:
err_rate = (C[1,2] + C[2,1]) / length(y)

0.07333333333333333

In [18]:
avg(y .!= yhat)

0.07333333333333333

## 14.3 多クラス識別器

### 多クラス識別の誤り率と混同行列

全体の誤り率は`avg(y .!= yhat)`で簡単に計算できる．$N$個の真値`y`と$N$個の予測`yhat`（値は$1,\ldots,K$のいずれか）が与えられたら，$K \times K$の混同行列を計算することができる．

In [19]:
error_rate(y, yhat) = avg(y .!= yhat);
function confusion_matrix(y, yhat, K)
C = zeros(K,K)
for i in 1:K for j in 1:K
    C[i,j] = sum((y .== i) .& (yhat .== j))
end end
return C
end;
# test for K=4 on random vectors of length 100
K = 4;
y = rand(1:K, 100); yhat = rand(1:K, 100);
C = confusion_matrix(y, yhat, K)

4×4 Array{Float64,2}:
 11.0   3.0   4.0  1.0
  4.0   7.0   7.0  4.0
  5.0   9.0  11.0  7.0
  7.0  10.0   4.0  6.0

In [21]:
using LinearAlgebra

In [22]:
error_rate(y, yhat),  1-sum(diag(C))/sum(C)

(0.65, 0.65)

関数`confusion_matrix`は`VMLS`パッケージに含まれている．


### 最小二乗多クラス識別器

（回帰モデルに基づく）$K$クラス識別器は次のように表せる．
$$
\hat{f}(x) = \mathop{\mathrm{argmax}}_{k=1,\ldots,K} \tilde{f}_k (x)
$$
ここで$\tilde{f}_k(x) = x^T \theta$である．$n$次元ベクトル$\theta_1, \ldots, \theta_k$はこのモデルのパラメータ（係数）である．
行列とベクトルの記法を使えば
$$
\hat{f}(x) = \mathop{\mathrm{argmax}} (x^T \Theta)
$$
とも書ける．ここで$\Theta=[\theta_1 \cdots \theta_K]$はモデル係数を表す$n \times K$行列である．この式のargmaxの意味は明らかだろう．

これをJuliaで書こう．関数`argmax(u)`は行ベクトルもしくは列ベクトル`u`の中の最大値のインデックス（つまり$\mathrm{argmax}_k u_k$）を求める．これを行列に拡張した関数`row_argmax`を定義する．これは，各行の最大値のインデックスをベクトルとして返す．

In [23]:
row_argmax(u) = [ argmax(u[i,:]) for i = 1:size(u,1) ];

In [24]:
A = randn(4,5)

4×5 Array{Float64,2}:
  0.84405     0.373452    0.269307   0.72903   -0.739056
 -2.48909    -0.769494   -2.01853   -0.330689   2.01277 
 -0.731525   -0.0217292  -1.46853    0.574108  -0.747028
 -0.0215695  -1.63563    -1.08885    1.65001   -2.03644 

In [25]:
row_argmax(A)

4-element Array{Int64,1}:
 1
 5
 4
 4

$N$個のサンプルからなるデータセットが$n \times N$のデータ行列`X`に保持されており，`Theta`が$係数ベクトル$\theta_k$を表すn \times K$行列とすると，以下の関数を定義できる．

```julia
fhat(X,Theta) = row_argmax(X'*Theta)
```

これは予測を$N$次元ベクトルとして返す．



### 行列最小二乗

クラス数が$K$，データセットのサンプル数が$N$，特徴量の数が$n$の場合の多クラス識別器の係数行列$\Theta$を最小二乗法で求めよう．データは$n \times N$行列$X$で与えられており，各サンプルのクラスを表す（要素が$1,\ldots,K$の）$N$次元ベクトルが$y^\mathrm{cl}$とする．最小二乗法の目的関数は以下のように行列ノルムの2乗で表される．
$$
\| X^T \Theta - Y \|^2
$$
ここで$Y$は$N \times K$行列で
$$
Y_{ij} =
\begin{cases}
\phantom{-}1 & y^\mathrm{cl} = j\\
-1 & y^\mathrm{cl} \neq j
\end{cases}
$$
である．つまり，$Y$の行はone-hot encodingで表されたクラス表現を$0/1$から$-1/+1$に変換したものである．最小二乗解は$\hat{\Theta} = (X^T)^\dagger Y$である．

これをJuliaで書いてみよう．

In [27]:
function one_hot(ycl,K)
N = length(ycl)
Y = zeros(N,K)
for j in 1:K
   Y[findall(ycl .== j), j] .= 1
end
return Y
end;

In [28]:
K = 4;
ycl = rand(1:K, 6)

6-element Array{Int64,1}:
 1
 1
 2
 2
 3
 1

In [29]:
Y = one_hot(ycl, K)

6×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0

In [30]:
2 * Y .- 1

6×4 Array{Float64,2}:
  1.0  -1.0  -1.0  -1.0
  1.0  -1.0  -1.0  -1.0
 -1.0   1.0  -1.0  -1.0
 -1.0   1.0  -1.0  -1.0
 -1.0  -1.0   1.0  -1.0
  1.0  -1.0  -1.0  -1.0

上で定義した関数を使うと，行列最少二乗問題による多クラス識別器が以下のように数行で実現できる．


In [38]:
function ls_multiclass(X, ycl, K)
n, N = size(X)
Theta = X' \ (2 * one_hot(ycl, K) .- 1)
yhat = row_argmax(X' * Theta)
return Theta, yhat
end;

### あやめの識別

あやめのデータセットのための3クラス識別器を作成する．150サンプルのデータセットを，120サンプルの訓練集合（各クラス40サンプル）と，30サンプルのテスト集合（各クラス10サンプル）に分割する．以下のコードでは上で定義した関数を呼び出している．


In [39]:
D = iris_data();
setosa = D["setosa"];
versicolor = D["versicolor"];
virginica = D["virginica"];
# 1, ..., 50についてのランダムな置換を3つ作成
import Random
I1 = Random.randperm(50);
I2 = Random.randperm(50);
I3 = Random.randperm(50);
# 訓練集合は各クラス40サンプルをランダムに用いる
Xtrain = [ setosa[I1[1:40],:];
           versicolor[I2[1:40],:];
           virginica[I3[1:40],:] ]'; # 4x120データ行列
# 定数1の特徴を追加
Xtrain = [ ones(1,120); Xtrain ];  # 5x120データ行列
ytrain = [ ones(40); 2*ones(40); 3*ones(40) ];
# テスト集合は各クラス残りの10サンプルを用いる
Xtest = [ setosa[I1[41:end],:];
          versicolor[I2[41:end],:]
          virginica[I3[41:end],:] ]'; # 4x30データ行列
Xtest = [ ones(1,30); Xtest ]; # 5x30データ行列
ytest = [ones(10); 2*ones(10); 3*ones(10)];
Theta, yhat = ls_multiclass(Xtrain, ytrain, 3);
Ctrain = confusion_matrix(ytrain, yhat, 3)

3×3 Array{Float64,2}:
 40.0   0.0   0.0
  0.0  27.0  13.0
  0.0   5.0  35.0

In [40]:
error_train = error_rate(ytrain, yhat)

0.15

In [41]:
yhat = row_argmax(Xtest' * Theta);
Ctest = confusion_matrix(ytest, yhat, 3)

3×3 Array{Float64,2}:
 10.0  0.0  0.0
  0.0  8.0  2.0
  0.0  3.0  7.0

In [42]:
error_test =  error_rate(ytest, yhat)

0.16666666666666666